In [1]:
# import packages
import pandas as pd
import os
from dotenv import load_dotenv
import mysql.connector

In [2]:
# Read the CSV file and specify numeric columns as strings
dtype_dict = {
    'customer_zip_code_prefix': str  
}

df = pd.read_csv('../data/raw/olist_customers_dataset.csv', dtype=dtype_dict)

In [3]:
df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,09790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,01151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,08775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  object
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: object(5)
memory usage: 3.8+ MB


In [5]:
df.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [6]:
# check for duplicate primary key
df['customer_id'].duplicated().sum()

0

In [7]:
# Create table, login and out of MySQL, and load data

# Load environment variables from .env file
load_dotenv()

# Get the password from the environment variable
db_password = os.getenv('DB_PASSWORD')

# Define the function
def create_and_load_table(connection, table_name, columns_def, df):
    cursor = connection.cursor()

    # Create table if it doesn't exist
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        {columns_def}
    )
    """
    cursor.execute(create_table_query)

    # Prepare the SQL query to insert data
    columns = ', '.join(df.columns)
    placeholders = ', '.join(['%s'] * len(df.columns))
    insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

    # Insert DataFrame values into MySQL table
    for index, row in df.iterrows():
        cursor.execute(insert_query, tuple(row))

    # Commit the transaction
    connection.commit()

    # Close the cursor
    cursor.close()

    
# Get the password from the environment variable
db_password = os.getenv('DB_PASSWORD')

if db_password is None:
    raise ValueError("DB_PASSWORD environment variable is not set")

connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password=db_password,
    database='olist_db'
)

# input: enter the the variable name for the desired dataframe to load
table_name = 'customers' #edit this line

# input: define the columns (edit the below)
columns_def = """
customer_id VARCHAR(200), 
customer_unique_id VARCHAR(200), 
customer_zip_code_prefix VARCHAR(50), 
customer_city VARCHAR(100), 
customer_state VARCHAR(10)
"""

# Call the function
create_and_load_table(connection, table_name, columns_def, df)

# Close the connection
connection.close()